In [1]:
!pip install ultralytics --quiet
!pip install -U ipywidgets --quiet
!pip uninstall wandb -y

!pip install mlflow --quiet
!pip install pyngrok --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.
Found existing installation: wandb 0.18.3
Uninstalling wandb-0.18.3:
  Successfully uninstalled wandb-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requir

In [2]:
import os
import shutil
from distutils.dir_util import copy_tree

import mlflow
from pyngrok import ngrok
from getpass import getpass

from ultralytics.data.utils import autosplit
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
get_ipython().system_raw("mlflow ui --port 5000 --backend-store-uri file:///kaggle/working/runs/mlflow &")

# Terminate open tunnels if exist
ngrok.kill()

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
ngrok_auth_token = user_secrets.get_secret("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(ngrok_auth_token)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)
mlflow.set_experiment(experiment_id="0")
mlflow.autolog()

MLflow Tracking UI: https://ce9a-35-226-7-253.ngrok-free.app                                        


In [4]:
from ultralytics import settings

# Update a setting
settings.update({"mlflow": True})

[2024-10-20 22:43:12 +0000] [132] [INFO] Starting gunicorn 23.0.0
[2024-10-20 22:43:12 +0000] [132] [INFO] Listening at: http://127.0.0.1:5000 (132)
[2024-10-20 22:43:12 +0000] [132] [INFO] Using worker: sync
[2024-10-20 22:43:12 +0000] [134] [INFO] Booting worker with pid: 134
[2024-10-20 22:43:12 +0000] [133] [INFO] Starting gunicorn 23.0.0
[2024-10-20 22:43:12 +0000] [133] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-10-20 22:43:12 +0000] [133] [ERROR] connection to ('127.0.0.1', 5000) failed: [Errno 98] Address already in use
[2024-10-20 22:43:12 +0000] [135] [INFO] Booting worker with pid: 135
[2024-10-20 22:43:12 +0000] [136] [INFO] Booting worker with pid: 136
[2024-10-20 22:43:12 +0000] [137] [INFO] Booting worker with pid: 137
[2024-10-20 22:43:13 +0000] [133] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2024-10-20 22:43:13 +0000] [133] [ERROR] connection to ('127.0.0.1', 5000) failed: [Errno 98] Address already in use
[2024-10-20 22:43:14 +0000] [133] [ERROR] Conne

In [5]:
# HYPERPARAMETERS
TRAIN_FRACTION = 0.8
VAL_FRACTION = 0.1
TEST_FRACTION = 0.1
EPOCHS = 200

assert ((TRAIN_FRACTION + VAL_FRACTION + TEST_FRACTION) == 1.0), "Sum of split fraction != 1"

In [6]:
# Copy train images from kaggle input directory to kaggle output directory
dataset_source_dir = '/kaggle/input/fixed-wing-uavs/dataset-07-10-2024/'
dataset_target_dir = '/kaggle/working/dataset/'

data_yaml_source_path = '/kaggle/working/dataset/data_kaggle.yaml'
data_yaml_target_path = '/kaggle/working/data_kaggle.yaml'

os.makedirs(dataset_target_dir, exist_ok=True)
copy_tree(dataset_source_dir, dataset_target_dir)
shutil.move(data_yaml_source_path, data_yaml_target_path)

'/kaggle/working/data_kaggle.yaml'

In [7]:
# create data split
autosplit(path="/kaggle/working/dataset", weights = (TRAIN_FRACTION, VAL_FRACTION, TEST_FRACTION))

Autosplitting images from /kaggle/working/dataset


100%|██████████| 891/891 [00:00<00:00, 19653.05it/s]


In [8]:
# use pre-trained yolov11 model
model = YOLO("yolo11n.pt")
results = model.train(data="/kaggle/working/data_kaggle.yaml",
                      epochs=2,
                      device=[0, 1])

Ultralytics 8.3.18 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/working/data_kaggle.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=None, name=train12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False

100%|██████████| 755k/755k [00:00<00:00, 13.9MB/s]
2024/10/20 22:44:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2024/10/20 22:44:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/dataset... 590 images, 132 backgrounds, 1 corrupt: 100%|██████████| 723/723 [00:00<00:00, 959.09it/s] 


train: WARNING ⚠️ /kaggle/working/dataset/517.jpg: ignoring corrupt image/label: cannot identify image file '/kaggle/working/dataset/517.jpg'
train: New cache created: /kaggle/working/dataset.cache


...

                   all         78         82        0.8        0.5      0.643      0.439

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        2/2      1.27G      1.183      2.001      1.414          1        640: 100%|██████████| 46/46 [00:09<00:00,  4.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  9.72it/s]


                   all         78         82      0.776      0.646       0.73      0.483

2 epochs completed in 0.009 hours.
Optimizer stripped from runs/detect/train12/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train12/weights/best.pt, 5.5MB

Validating runs/detect/train12/weights/best.pt...
Ultralytics 8.3.18 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:01<00:00,  4.81it/s]


                   all         78         82      0.776      0.646      0.729      0.482
Speed: 0.2ms preprocess, 3.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/train12
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


In [9]:
import random

def generate_random_image_paths(num_paths):
    with open("/kaggle/working/autosplit_test.txt", "r") as f:
        data = f.read().replace('./', '/kaggle/working/').split('\n')
        data = data[:-1]
    
    generated_paths = []
    image_nums = random.sample(range(0, len(data)), num_paths)

    for i in image_nums:
        generated_paths.append(data[i])
        
    return generated_paths

In [ ]:
paths = generate_random_image_paths(60)
print(paths)

In [ ]:
model = YOLO('/kaggle/working/runs/detect/different_split/weights/best.pt')
results = model(paths, batch=16, device=0,save=True, project='fixed-wing-uavs', name='results')
for result in results:
    result.show()  # display to screen
    